# 6.1 Data Loaders and Splitters

학습 목표: RAG의 첫번째 단계인 Retrieval에 대해 알아본다.

Retrival은 랭체인의 모듈이다. 어떻게 작동하는지 알아보자.

1. 여러 종류의 Source데이터가 있다.
2. 그다음에 그 데이터를 불러올 수 있는 Loader가 있다. 랭체인에는 많은 Document loader가 있고, 여러 곳에서 데이터를 불러오는 많은 Intergration들이 있는데 우리는 그 데이터를 로드 할 것이다.
3. 그 다음엔 불러온 데이터를 변환할 것이다. 정확히 말하자면, 데이터를 분할할 것이다. 나중에 데이터를 임베드하기 위해 데이터를 분할한다고 생각하면 된다. 여기서 '임베드'는 다음 6.2에서 배워 볼 것이다.
4. 그리고 임베드를 저장할 것인데, 어디에 저장할지도 다음에 배운다.

이게 RAG의 첫번째 단계인 데이터를 Retrieve하는 과정이다.

위의 설명은 다음의 사진에 서 잘 설명되어 있다.
```python
"./RetrieverDataConnection.png"
```

이번에는 데이터를 로드해서 분할하는 것까지 배운다.

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader

llm = ChatOpenAI(temperature=0.1)

loaderTxt = TextLoader("../files/chapter_one.txt")
loaderPdf = PyPDFLoader("../files/chapter_one.pdf")

print(loaderTxt.load())
print("\n")
print(loaderPdf.load())

[Document(page_content='**Title: The Clockmaker\'s Masquerade**\n\n---\n\n### **I. The Curious Discovery**\n\nThe workshop of Oldwyn Rook was a place of quiet, tinkling sounds and the smell of clockwork oil. Tucked away in the corner of a nameless cobblestone street, the dilapidated little building barely stood out from the surrounding gray stone facades. However, once you stepped inside, it was an entirely different universe-a realm where tiny gears and cogs danced beneath the slender fingers of an eccentric genius. \n\nOldwyn, a lanky man with a mane of wild gray hair, peered through round-rimmed glasses that magnified his inquisitive blue eyes. He was famed in the district for crafting timepieces of such delicate intricacy that you could swear you heard their hearts beating. Yet his best-kept secret was something far more extraordinary than a mere watch or clock. \n\nLate one evening, while rummaging through the battered trunk in his dusty storage room, Oldwyn discovered a peculiar 

이렇게 각각의 파일 형식에 맞는 Loader를 임포트 해와도 되지만 UnstructuredFileLoader를 사용하면 더 편리하다.

In [7]:
from langchain.document_loaders import UnstructuredFileLoader

loaderTxt = UnstructuredFileLoader("../files/chapter_one.txt")
loaderPdf = UnstructuredFileLoader("../files/chapter_one.pdf")
loaderDocx = UnstructuredFileLoader("../files/chapter_one.docx")

print(loaderTxt.load())
print(len(loaderTxt.load()))
print("\n")
print(loaderPdf.load())
print(len(loaderPdf.load()))
print("\n")
print(loaderDocx.load())
print(len(loaderDocx.load()))

[Document(page_content='\n\n\n\nTitle: The Clockmaker\'s Masquerade**\n\n---\n\n### *\n\n\n\nI. The Curious Discovery**\n\nThe workshop of Oldwyn Rook was a place of quiet, tinkling sounds and the smell of clockwork oil. Tucked away in the corner of a nameless cobblestone street, the dilapidated little building barely stood out from the surrounding gray stone facades. However, once you stepped inside, it was an entirely different universe-a realm where tiny gears and cogs danced beneath the slender fingers of an eccentric genius.\n\nOldwyn, a lanky man with a mane of wild gray hair, peered through round-rimmed glasses that magnified his inquisitive blue eyes. He was famed in the district for crafting timepieces of such delicate intricacy that you could swear you heard their hearts beating. Yet his best-kept secret was something far more extraordinary than a mere watch or clock.\n\nLate one evening, while rummaging through the battered trunk in his dusty storage room, Oldwyn discovered 

보다시피 잘 작동한다.

이제 데이터를 분할할 것이다. 분할하는 이유는 loader.load()의 리턴값을 보면 Document로 이루어진 리스트인데, 이 경우에는 전체 챕터가 하나의 문서에 들어가 있다. 즉 너무 큰 덩어리이다. len메서드로 리스트의 요소가 몇개인지 확인해보면 1개라는 것을 알 수 있다. 즉 모든 텍스트가 1개의 요소안에 들어가 있는 것이다.

문서가 너무 큰 덩어리이기 때문에, 나누는 작업을 해야한다.

우리가 문서를 임베드 또는 저장하거나 언어 모델에게 주고 싶다면, 질문에 답해야 할 때 필요한 '파일의 부분들'만을 전달 할 수 있다. 그래서 우리는 이 파일을 조각들로 나눠야 한다.

즉, 이 문서를 조각들로 나누면 필요한 것을 찾기 더 쉬워진다. 그럼 우리가 만들 Prompt도 짧아지게 된다. 왜냐하면 전체 문서나 챕터를 LLM에게 주는게 아니라, 딱 필요한 부분만 쓰는것이기 때문이다.

분할하는 방법에도 여러가지가 있다. 일반적인 방법이 있고 고급적인 방법도 있다. 우리는 chapGPT 3,4,OpenAI와 잘 작동하는 것을 사용할 것이다. OpenAI의 실제 함수를 사용해서 그렇게 할 것이다.

이제 아주 일반적인 Text Spliter를 사용해보자.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter()
# 이 RecursiveCharacterTextSplitter는 우리의 파일을 분할해주는데, 문장의 끝이나 문단의 끝부분마다 끊어준다.
# 문장의 중간에서는 끊지 않는다.
# splitter를 가져온 후, 두가지 옵션이 있다.

loader = UnstructuredFileLoader("../files/chapter_one.docx")

# 첫번째 옵션
docs = loader.load()

splitted_docs = splitter.split_documents(docs)
#이렇게 하는게 첫번째 옵션
print(splitted_docs)
print(len(splitted_docs))
print("\n")


#두번째 옵션
splitted_docs = loader.load_and_split(text_splitter=splitter)
# 이렇게 하는게 두번째 옵션
print(splitted_docs)
print(len(splitted_docs))
print("\n")

[Document(page_content='**Title: The Clockmaker’s Masquerade**\n\n---\n\n### **I. The Curious Discovery**\n\nThe workshop of Oldwyn Rook was a place of quiet, tinkling sounds and the smell of clockwork oil. Tucked away in the corner of a nameless cobblestone street, the dilapidated little building barely stood out from the surrounding gray stone facades. However, once you stepped inside, it was an entirely different universe—a realm where tiny gears and cogs danced beneath the slender fingers of an eccentric genius. \n\nOldwyn, a lanky man with a mane of wild gray hair, peered through round-rimmed glasses that magnified his inquisitive blue eyes. He was famed in the district for crafting timepieces of such delicate intricacy that you could swear you heard their hearts beating. Yet his best-kept secret was something far more extraordinary than a mere watch or clock. \n\nLate one evening, while rummaging through the battered trunk in his dusty storage room, Oldwyn discovered a peculiar m

이렇게 분할하는 방법에는 두가지가 있었고, 문장이나 문단을 기준으로 데이터를 분할했다. 아주 좋은 기능이다. 그런데 너무 짧거나 길게 분할되었다고 판단되면 이 분할되는 사이즈를 조절할 수 있다. "chunk_size"를 설정해주면 된다.

In [9]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
)

splitted_docs = loader.load_and_split(text_splitter=splitter)

print(splitted_docs)
print(len(splitted_docs))

[Document(page_content='**Title: The Clockmaker’s Masquerade**\n\n---\n\n### **I. The Curious Discovery**', metadata={'source': '../files/chapter_one.docx'}), Document(page_content='The workshop of Oldwyn Rook was a place of quiet, tinkling sounds and the smell of clockwork oil. Tucked away in the corner of a nameless cobblestone street, the dilapidated little building barely stood out from the surrounding gray stone facades. However, once you stepped inside, it was an', metadata={'source': '../files/chapter_one.docx'}), Document(page_content='oil. Tucked away in the corner of a nameless cobblestone street, the dilapidated little building barely stood out from the surrounding gray stone facades. However, once you stepped inside, it was an entirely different universe—a realm where tiny gears and cogs danced beneath the slender fingers of', metadata={'source': '../files/chapter_one.docx'}), Document(page_content='stood out from the surrounding gray stone facades. However, once you steppe

그러나 이렇게 chunk_size를 활용해서 데이터의 사이즈를 조절하게되면 문장이 중간에 끊겨서 파괴되게 된다. 그래서 문장의 의미를 파악할 수 없게 된다. 그래서 좋은 방법이 아니다.

그러나 우리는 여전히 작은 조각들이 필요하다. 그래서 Chunk Overlap이라는 속성을 쓴다. 이 속성은 문장이나 문단을 분할할 때 앞 조각 일부분을 가져오게 만든다. 그러니까 중간에서 자르는게 아니라 앞 조각의 끝 부분을 조금 가져와서 다음 조각에 연결시키는 방식으로 작동한다. 이렇게 되면 문서 사이에 겹치는 부분이 생기게 되지만 이런식으로 작동된다.

In [10]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
)

splitted_docs = loader.load_and_split(text_splitter=splitter)

print(splitted_docs)
print(len(splitted_docs))

[Document(page_content='**Title: The Clockmaker’s Masquerade**\n\n---\n\n### **I. The Curious Discovery**', metadata={'source': '../files/chapter_one.docx'}), Document(page_content='The workshop of Oldwyn Rook was a place of quiet, tinkling sounds and the smell of clockwork oil. Tucked away in the corner of a nameless cobblestone street, the dilapidated little building barely stood out from the surrounding gray stone facades. However, once you stepped inside, it was an', metadata={'source': '../files/chapter_one.docx'}), Document(page_content='However, once you stepped inside, it was an entirely different universe—a realm where tiny gears and cogs danced beneath the slender fingers of an eccentric genius.', metadata={'source': '../files/chapter_one.docx'}), Document(page_content='Oldwyn, a lanky man with a mane of wild gray hair, peered through round-rimmed glasses that magnified his inquisitive blue eyes. He was famed in the district for crafting timepieces of such delicate intricacy 

다른 Splitter도 있다. 바로 Character Text Splitter이다. 얘도 작동방식은 비슷하다. chunk_size, chunk_overlap등이 있는것은 같다. 그러고 얘는 하나가 더 있다. separator라는 속성이다. separator는 특정 문자열을 찾은 다음에 거기부터 끊는 역할을 한다. 실습을 통해 알아보자

In [12]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=50,
)

splitted_docs = loader.load_and_split(text_splitter=splitter)

print(splitted_docs)
print(len(splitted_docs))

[Document(page_content='**Title: The Clockmaker’s Masquerade**\n---\n### **I. The Curious Discovery**\nThe workshop of Oldwyn Rook was a place of quiet, tinkling sounds and the smell of clockwork oil. Tucked away in the corner of a nameless cobblestone street, the dilapidated little building barely stood out from the surrounding gray stone facades. However, once you stepped inside, it was an entirely different universe—a realm where tiny gears and cogs danced beneath the slender fingers of an eccentric genius.', metadata={'source': '../files/chapter_one.docx'}), Document(page_content='Oldwyn, a lanky man with a mane of wild gray hair, peered through round-rimmed glasses that magnified his inquisitive blue eyes. He was famed in the district for crafting timepieces of such delicate intricacy that you could swear you heard their hearts beating. Yet his best-kept secret was something far more extraordinary than a mere watch or clock.', metadata={'source': '../files/chapter_one.docx'}), Doc